In [53]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import pandas as pd

In [17]:
class Node:
    def __init__(self,
                 left = None,
                 right = None,
                 prediction = None,
                 numSamples = None,
                 featureSplit = None,
                 threshold = None,
                 samples = None
                ):
        
        self.left = left
        self.right = right  
        self.prediction = prediction
        self.featureSplit = featureSplit 
        self.threshold = threshold
        self.samples = samples
        
class RegressionTree:
    
    def __init__(self, maxDepth = 10, minSamples = 2, numFeatures = None):
        self.maxDepth = maxDepth
        self.minSamples = minSamples
        self.root = None
        self.numFeatures = None
    
    def fit(self, X, y):
        #recursively grows tree
        self.numFeatures = X.shape[1] if not self.numFeatures else min(X.shape[1], self.numFeatures)
        self.root = self.__growTree(X, y)
        
    def predict(self, X):
        #traverses fit tree for every observation in prediction set
        return [self.__traverseTree(x, self.root) for x in X]
    
    def __growTree(self, X, y, depth=0):

        numSamples, totalNumFeatures = X.shape
        
        #check stopping criteria
        if depth >= self.maxDepth or numSamples <= self.minSamples or len(set(y))==1:
            return Node(prediction = np.mean(y), samples = X)
        
        #find best split
        bestFeature, bestThreshold, SSR = self.__bestSplit(X, y, totalNumFeatures)
        
        #create child nodes based on best determined split
        leftSplit, rightSplit = self.__split(X[:,bestFeature], bestThreshold)
        
        #continue growing tree
        leftNode = self.__growTree(X[leftSplit], y[leftSplit], depth=depth+1)
        rightNode = self.__growTree(X[rightSplit], y[rightSplit], depth=depth+1)
    
        return Node(left= leftNode,
                    right=rightNode,
                    featureSplit = bestFeature,
                    threshold = bestThreshold,
                    samples = X
                   )

    
    def __bestSplit(self, X, y, totalNumFeatures):
        
        minSSR = None
        
        #iteratively run through all possible features, thresholds and find best split (smallest SSR)
        for feature in self.__getFeatures(totalNumFeatures):
            featureData = X[:, feature]
            for threshold in self.__getThresholds(featureData):
                #calculate prediction (average of y) and SSR for current feature, threshold
                SSR = self.__getSplitSSR(y, featureData, threshold)
                
                #store feature, threshold that produces the smallest error
                if not minSSR or SSR < minSSR:
                    minSSR = SSR
                    bestFeature, bestThreshold = feature, threshold
        
        return  bestFeature, bestThreshold, minSSR 
    
    def __getFeatures(self, totalNumFeatures):
        return np.random.choice(totalNumFeatures, self.numFeatures, replace=False)
    
    def __getThresholds(self, featureData):
        #given a set of continuous values, determines possible thresholds to split values on (using midpoint of adjacent values)
        allFeatureValues = np.sort(np.unique(featureData))
        thresholds = []
        for i in range(1, len(allFeatureValues)):
            thresholds.append((allFeatureValues[i]+allFeatureValues[i-1])/2)
        
        return thresholds
    
    
    def __getSplitSSR(self, y, featureData, threshold):        
        #create children
        leftNode, rightNode = self.__split(featureData, threshold)
        
        #caculate SSR
        SSRLeft = self.__ssr(y[leftNode])
        SSRRight = self.__ssr(y[rightNode])     
        
        return SSRLeft + SSRRight
    
    def __ssr(self, y):
        prediction = np.mean(y)
        return sum(np.square(y - prediction))
        
    def __split(self, featureData, threshold):
        #splits observations based on a given feature and threshold
        leftNode, rightNode = np.where(featureData <= threshold), np.where(featureData > threshold)
        return leftNode, rightNode
    
    def __traverseTree(self, x, node):
        #given an observation X, traverses through tree to determine its prediction
        
        if node.prediction is not None: #if node is a leaf node, return prediction. else continue running through tree
            return node.prediction
        
        #given the feature and threshold that is used by the current node, evaluate whether to continue to the left/right of the node 
        elif x[node.featureSplit] <= node.threshold:
            return self.__traverseTree(x, node.left)
        
        elif x[node.featureSplit] > node.threshold:
            return self.__traverseTree(x, node.right)
    

In [74]:
X, y = datasets.make_regression(n_samples=500, noise=0, random_state=123, n_features=5)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state=123)

In [75]:
model = RegressionTree()
model.fit(X_train,  Y_train)
predictions = model.predict(X_test)

In [80]:
mean_squared_error( Y_test, predictions, squared=False)

52.517263847339976

In [83]:
# fig = plt.figure(figsize=(10,6))
# plt.scatter(X_test[:,1], Y_test, color='green')
# plt.scatter(X_test[:,1], predictions, color='black', linewidth=1)